In [205]:
print('sss')

sss


In [206]:
import pandas as pd
import numpy as np

In [207]:
## read the file
path=r'C:\Users\arvinmvn\Desktop\Ranking\Ranking_IN_Sample_Output4.xlsx'
df=pd.read_excel(path)

df.head()

,provider_id,service_name
0,4f796148-38fc-441d-857e-c8db05ca6309,Excess Inventory
1,901aa6b4-267f-483b-b6b1-bb6a606452ea,Storage
2,3b1b5637-fe6d-4b3a-bed8-915a6662862d,Accounting
3,ef573518-fc0c-4f61-82d3-6898d0f73484,Advertising Optimization
4,0ff82c94-f00a-4ce3-ad97-90525a5a2d23,Seller Reinstatement


We will create a view that has provider id, service name,Standard Metrics 1,Standard Metrics 2,Standard Metrics 3,Standard Metrics 4,Standard Metrics 5,Standard Metrics 6,Service Metrics 1,Service Metrics 2

after that we will assign the weights and calculate scores by creating blocks for each service

lets take individual sheets 1st, each sheet has provider id vs service vs metric, we will take it in different data frames and go a join on provider_id and service

In [208]:
provider_list=pd.read_excel(path,sheet_name='provider_list')

In [209]:
sp_onboarding_details=pd.read_excel(path,sheet_name='sp_onboarding_details')
sp_onboarding_details['displayname']=sp_onboarding_details['displayname'].astype(str)
sp_onboarding_details=sp_onboarding_details[['final_providerid','displayname','onboarding_date','updated_date']]

In [210]:
#sp_onboarding_details['displayname'].astype(str).dtype

In [211]:
ytc_percent_in_week=pd.read_excel(path,sheet_name='YTC % (In week)')
#ytc_percent_in_week=ytc_percent_in_week[['providerid','service','ytc_in_week_percent']]

In [212]:
ytc_sla_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC SLA (LTD 21-7 days)')
ytc_sla_ltd_21_7_days=ytc_sla_ltd_21_7_days[['providerid','service','ytc_sla_21_7']]

In [213]:
ytc_percent_ltd_21_7_days=pd.read_excel(path,sheet_name='YTC % (LTD 21 - 7 days)')
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[ytc_percent_ltd_21_7_days['marketplaceid'].notnull()]
ytc_percent_ltd_21_7_days=ytc_percent_ltd_21_7_days[['providerid','service','ytc_ltd_21_7_percent']]

In [214]:
snr_orders_completed=pd.read_excel(path,sheet_name='snr_orders(completed)')
snr_orders_completed['service']='Seller Reinstatement'

In [215]:
cat_orders_completed=pd.read_excel(path,sheet_name='cat_orders(completed)')
cat_orders_completed['service']='Cataloguing'

In [216]:
orders_completed=pd.read_excel(path,sheet_name='Orders(completed)')

## Add SNR orders

## we will replace the services keywords from orders query so that join is mapped correctly, we will also add snr into this

orders_completed=orders_completed.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )

orders_completed=pd.concat([orders_completed[['providerid','service','total_4w_orders']],
           snr_orders_completed[snr_orders_completed['metric']=='No. of orders'][['providerid','service','orders']].rename(columns = {'orders': 'total_4w_orders'}, inplace = False)
          ])

## add calalog orders
orders_completed=pd.concat([orders_completed,cat_orders_completed.rename(columns = {'no_orders': 'total_4w_orders'}, inplace = False)])

In [217]:
avg_feedback=pd.read_excel(path,sheet_name='avg_feedback')

In [218]:
unique_seller=pd.read_excel(path,sheet_name='unique_seller')
unique_seller=unique_seller.replace( ['Advertise','Boost','FBA','Enhanced Brand Content'], ['Advertising Optimization','Account Management','FBA Preparation','Enhanced Brand Content'] )
unique_seller=pd.concat([unique_seller,snr_orders_completed[snr_orders_completed['metric']=='No. of sellers'][['providerid','service','orders']].rename(columns = {'orders': 'adoption'}, inplace = False)
          ])        

In [219]:
pip_percent=pd.read_excel(path,sheet_name='pip_%')

pip_percent=pip_percent[['providerid','service','pip_percent']]

In [220]:
caq=pd.read_excel(path,sheet_name='CAQ')
caq['service']='Cataloguing'

In [221]:
cat_asins_created=pd.read_excel(path,sheet_name='cat_asins_created')
cat_asins_created['service']='Cataloguing'

In [222]:
ad_revenue=pd.read_excel(path,sheet_name='Ad Revenue')
ad_revenue['service']='Advertising Optimization'

#ad_revenue=ad_revenue.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()
ad_revenue=ad_revenue[['providerid','total_spend','service']]

In [223]:
## we are the 4 weeks of acos data and doing an average of them

## we are doing reset index to ensure that the df is not getting converted to a series.
ad_acos=pd.read_excel(path,sheet_name='ad_ACOS')
ad_acos['service']='Advertising Optimization'
#ad_acos=ad_acos.groupby(['providerid','service'])['sp_revenue','attributed_gms','budget','gms','ad_revenue','acos_p'].sum().reset_index()

ad_acos['acos_p']=ad_acos['numeraor']/ad_acos['denominator']
ad_acos['acos_p']=ad_acos['acos_p'].replace(np.inf,0)
ad_acos = ad_acos.groupby(['providerid','service'])['acos_p'].mean().reset_index()
ad_acos=ad_acos[['providerid','acos_p','service']]


In [224]:
AM_gms=pd.read_excel(path,sheet_name='AM_gms')
AM_gms['service']='Account Management'
AM_gms=AM_gms.rename(columns = {'gms': 'boost_gms'}, inplace = False)

In [225]:
ebc_gv=pd.read_excel(path,sheet_name='ebc_gv')
ebc_gv['service']='Enhanced Brand Content'
ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()
ebc_gv=ebc_gv.rename(columns = {'asins_created': 'ebc_gv_asins_created'}, inplace = False)

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_13848\820171744.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ebc_gv=ebc_gv.groupby(['provider_id','service'])['asins_created','lw_gv_platform','lw_gv_3p','lw_gv_3p_total','lw_gv_platform_total','spn_flag','lw_gv_3p_ytd','lw_gv_platform_ytd','lw_gv_3p_total','lw_gv_platform_total'].sum().reset_index()


In [226]:
ebc_asin_coverage=pd.read_excel(path,sheet_name='ebc_asin_coverage')
ebc_asin_coverage['service']='Enhanced Brand Content'
ebc_asin_coverage=ebc_asin_coverage.groupby(['provider_id','service'])['asins_created'].sum().reset_index()
ebc_asin_coverage=ebc_asin_coverage.rename(columns = {'asins_created': 'ebc_asin_coverage_asins_created'}, inplace = False)

In [227]:
tax_unblocked_ttm=pd.read_excel(path,sheet_name='tax_unblocked_ttm')
tax_unblocked_ttm['service']='Taxes'
tax_unblocked_ttm=tax_unblocked_ttm.rename(columns = {'unblocked_gms': 'tax_unblocked_gms'}, inplace = False)

In [228]:
SNR_reinstatement_p_num=pd.read_excel(path,sheet_name='SNR_reinstatement_p_num')

In [229]:
SNR_reinstatement_p_deno=pd.read_excel(path,sheet_name='SNR_reinstatement_p_deno')

In [230]:

## filter out numerator and denomenator from the respective blocks and do a join on provider id vs num vs deno
snr_reinstatement_p=pd.merge(
    SNR_reinstatement_p_num[['providerid','no_sellers']].rename(columns = {'no_sellers': 'snr_reinst_p_num'}, inplace = False),
    SNR_reinstatement_p_deno[['providerid','ytd']].rename(columns = {'ytd': 'snr_reinst_p_deno'}, inplace = False),
    how='outer', 
    left_on =['providerid'],
    right_on=['providerid']
    )
## create a view on providerid vs (num/deno) and fill na with o
snr_reinstatement_p['snr_reinst_p']=snr_reinstatement_p['snr_reinst_p_num']/snr_reinstatement_p['snr_reinst_p_deno']
snr_reinstatement_p=snr_reinstatement_p.fillna(0).round(4)
snr_reinstatement_p['service']='Seller Reinstatement'
snr_reinstatement_p=snr_reinstatement_p[['providerid','snr_reinst_p','service']]

In [231]:
ds_ncns=pd.read_excel(path,sheet_name='DS_NCNS')
ds_ncns['service']='Domestic Shipping'

In [232]:
compliance_unblocked_gms=pd.read_excel(path,sheet_name='compliance_unblocked_gms')
compliance_unblocked_gms['service']='Compliance'
compliance_unblocked_gms=compliance_unblocked_gms.rename(columns = {'unblocked_gms': 'compliance_unblocked_gms'}, inplace = False)

In [233]:
join_1_2=pd.merge(
    provider_list,
    ytc_percent_in_week.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False),
    how='left',
    left_on=['provider_id','service_name'],
    right_on=['providerid_1','service_1']
    )

In [234]:
len(join_1_2)

6027

In [235]:
join_1_2.columns

Index(['provider_id', 'service_name', 'providerid_1', 'service_1',
       'ytc_in_week_numerator', 'ytc_in_week_denomenator',
       'ytc_in_week_percent'],
      dtype='object')

In [236]:
#ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_1', 'service': 'service_1'}, inplace = False)

In [237]:
join_1_3=pd.merge(
    join_1_2.drop(['providerid_1','service_1'],axis=1, inplace=False),
    ytc_sla_ltd_21_7_days.rename(columns = {'providerid': 'providerid_2', 'service': 'service_2'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_2','service_2']
    )

In [238]:
len(join_1_3)

6027

In [239]:
join_1_3.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'providerid_2',
       'service_2', 'ytc_sla_21_7'],
      dtype='object')

In [240]:
join_1_4=pd.merge(
    join_1_3.drop(['providerid_2','service_2'],axis=1, inplace=False),
    ytc_percent_ltd_21_7_days.rename(columns = {'providerid': 'providerid_3', 'service': 'service_3'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_3','service_3']
    )

In [241]:
join_1_4.to_clipboard()

In [242]:
len(join_1_4)

6027

In [243]:
join_1_5=pd.merge(
    join_1_4.drop(['providerid_3','service_3'],axis=1, inplace=False),
    orders_completed.rename(columns = {'providerid': 'providerid_4', 'service': 'service_4'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_4','service_4']
    )

In [244]:
len(join_1_5)

6027

In [245]:
join_1_6=pd.merge(
    join_1_5.drop(['providerid_4','service_4'],axis=1, inplace=False),
    cat_orders_completed.rename(columns = {'providerid': 'providerid_5', 'service': 'service_5'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_5','service_5']
    )

In [246]:
len(join_1_6)

6027

In [247]:
snr_reinstatement_p.columns

Index(['providerid', 'snr_reinst_p', 'service'], dtype='object')

In [248]:
join_1_7=pd.merge(
    join_1_6.drop(['providerid_5','service_5'],axis=1, inplace=False),
    snr_reinstatement_p.rename(columns = {'providerid': 'providerid_6', 'service': 'service_6'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_6','service_6']
    )

In [249]:
len(join_1_7)

6027

In [250]:
join_1_8=pd.merge(
    join_1_7.drop(['providerid_6','service_6'],axis=1, inplace=False),
    avg_feedback.rename(columns = {'provider_id': 'providerid_7', 'services_applicable': 'service_7'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_7','service_7']
    )

In [251]:
len(join_1_8)

6027

In [252]:
join_1_9=pd.merge(
    join_1_8.drop(['providerid_7','service_7'],axis=1, inplace=False),
    unique_seller.rename(columns = {'providerid': 'providerid_8', 'service': 'service_8'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_8','service_8']
    )

In [253]:
len(join_1_9)

6027

In [254]:
join_1_10=pd.merge(
    join_1_9.drop(['providerid_8','service_8'],axis=1, inplace=False),
    pip_percent.rename(columns = {'providerid': 'providerid_9', 'service': 'service_9'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_9','service_9']
    )

In [255]:
len(join_1_10)

6027

In [256]:
join_1_11=pd.merge(
    join_1_10.drop(['providerid_9','service_9'],axis=1, inplace=False),
    caq.rename(columns = {'providerid': 'providerid_10', 'service': 'service_10'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_10','service_10']
    )

In [257]:
len(join_1_11)

6027

In [258]:
join_1_12=pd.merge(
    join_1_11.drop(['providerid_10','service_10'],axis=1, inplace=False),
    cat_asins_created.rename(columns = {'provider_name': 'providerid_11', 'service': 'service_11'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_11','service_11']
    )

In [259]:
len(join_1_12)

6027

In [260]:
join_1_13=pd.merge(
    join_1_12.drop(['providerid_11','service_11'],axis=1, inplace=False),
    ad_revenue.rename(columns = {'providerid': 'providerid_12', 'service': 'service_12'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_12','service_12']
    )

In [261]:
len(join_1_13)

6027

In [262]:
join_1_14=pd.merge(
    join_1_13.drop(['providerid_12','service_12'],axis=1, inplace=False),
    ad_acos.rename(columns = {'providerid': 'providerid_13', 'service': 'service_13'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_13','service_13']
    )

In [263]:
len(join_1_14)

6027

In [264]:
join_1_15=pd.merge(
    join_1_14.drop(['providerid_13','service_13'],axis=1, inplace=False),
    AM_gms.rename(columns = {'providerid': 'providerid_14', 'service': 'service_14'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_14','service_14']
    )

In [265]:
len(join_1_15)

6027

In [266]:
join_1_16=pd.merge(
    join_1_15.drop(['providerid_14','service_14'],axis=1, inplace=False),
    ebc_gv.rename(columns = {'provider_id': 'providerid_15', 'service': 'service_15'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_15','service_15']
    )

In [267]:
len(join_1_16)

6027

In [268]:
join_1_17=pd.merge(
    join_1_16.drop(['providerid_15','service_15'],axis=1, inplace=False),
    ebc_asin_coverage.rename(columns = {'provider_id': 'providerid_16', 'service': 'service_16'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_16','service_16']
    )

In [269]:
len(join_1_17)

6027

In [270]:
join_1_18=pd.merge(
    join_1_17.drop(['providerid_16','service_16'],axis=1, inplace=False),
    tax_unblocked_ttm.rename(columns = {'providerid': 'providerid_17', 'service': 'service_17'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_17','service_17']
    )

In [271]:
len(join_1_18)

6027

In [272]:
join_1_19=pd.merge(
    join_1_18.drop(['providerid_17','service_17'],axis=1, inplace=False),
    ds_ncns.rename(columns = {'provider_id': 'providerid_18', 'service': 'service_18'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_18','service_18']
    )

In [273]:
len(join_1_19)

6027

In [274]:
join_1_20=pd.merge(
    join_1_19.drop(['providerid_18','service_18'],axis=1, inplace=False),
    compliance_unblocked_gms.rename(columns = {'providerid': 'providerid_19', 'service': 'service_19'}, inplace = False),
    how='left', 
    left_on =['provider_id','service_name'],
    right_on=['providerid_19','service_19']
    )

In [275]:
len(join_1_20)

6027

In [276]:
join_1_21=pd.merge(
    join_1_20.drop(['providerid_19','service_19'],axis=1, inplace=False),
    sp_onboarding_details.rename(columns = {'final_providerid': 'spid'}, inplace = False),
    how='left', 
    left_on =['provider_id'],
    right_on=['spid']
    )

In [277]:
len(join_1_21)

6027

In [278]:
join_1_21.to_clipboard()

In [279]:
join_1_21.columns

Index(['provider_id', 'service_name', 'ytc_in_week_numerator',
       'ytc_in_week_denomenator', 'ytc_in_week_percent', 'ytc_sla_21_7',
       'ytc_ltd_21_7_percent', 'total_4w_orders', 'no_orders', 'snr_reinst_p',
       'avg_feedback', 'adoption', 'pip_percent', 'caq_score',
       'count_of_asins', 'total_spend', 'acos_p', 'boost_gms',
       'ebc_gv_asins_created', 'lw_gv_platform', 'lw_gv_3p', 'lw_gv_3p_total',
       'lw_gv_platform_total', 'spn_flag', 'lw_gv_3p_ytd',
       'lw_gv_platform_ytd', 'lw_gv_3p_total', 'lw_gv_platform_total',
       'ebc_asin_coverage_asins_created', 'tax_unblocked_gms', 'ncns',
       'unblocked_asin', 'compliance_unblocked_gms', 'spid', 'displayname',
       'onboarding_date', 'updated_date'],
      dtype='object')

In [280]:
## we have the metrics view prepared, few niggles that need to be checked are
# provider id list for cataloging for those which has that spn codes

In [281]:
## Ranking view for imaging

In [282]:
## let's have the weights for imaging in this block

ytc_percent_in_week_percent_imaging_wt= 10/100
ytc_sla_ltd_21_7_days_imaging_wt= 5/100
ytc_percent_ltd_21_7_days_imaging_wt= 5/100
pip_percent_imaging_wt= 10/100
orders_completed_imaging_wt= 35/100
unique_seller_imaging_wt= 15/100
avg_feedback_imaging_wt= 20/100


In [283]:
## imaging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

imag_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

imag_rank=imag_rank[imag_rank['service_name']=='Imaging']
imag_rank=imag_rank.fillna(0)

##prepare the score

imag_rank['score']=100*(  (ytc_percent_in_week_percent_imaging_wt*imag_rank['ytc_in_week_percent']/imag_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
                    + (ytc_sla_ltd_21_7_days_imaging_wt*imag_rank['ytc_sla_21_7']/imag_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
                    + (ytc_percent_ltd_21_7_days_imaging_wt*imag_rank['ytc_ltd_21_7_percent']/imag_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    + (pip_percent_imaging_wt*imag_rank['pip_percent']/imag_rank['pip_percent'].max()).replace(np.nan,0)
                    + (orders_completed_imaging_wt*imag_rank['total_4w_orders']/imag_rank['total_4w_orders'].max()).replace(np.nan,0)
                    + (unique_seller_imaging_wt*imag_rank['adoption']/imag_rank['adoption'].max()).replace(np.nan,0)
                    + (avg_feedback_imaging_wt*imag_rank['avg_feedback']/imag_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

imag_rank['rank']=imag_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
imag_rank = imag_rank.sort_values(by="rank", ignore_index=True)
imag_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,0.00,0.76,0.0,47.83,2077.0,75.0,5.00,2021-08-12,2018-10-08,Value For Sellers,61.130662,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,0.18,0.00,0.0,44.44,1227.0,0.0,4.94,2023-02-09,2018-08-26,Sellers Cab,44.898456,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,0.00,0.00,0.0,65.22,972.0,1.0,5.00,2022-10-14,2019-04-02,Lynex Global Solutions,42.918856,3.0
3,860468c6-9e94-4627-8e89-23e58e944fff,Imaging,1.20,0.00,0.0,43.33,181.0,859.0,4.90,2023-01-05,2018-08-27,e-Vitamins,42.103072,4.0
4,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Imaging,0.00,0.00,0.0,30.00,806.0,9.0,4.82,2023-02-15,2020-01-09,Professional Hub,36.019249,5.0


In [284]:
## let's have the weights for Accounting in this block

ytc_percent_in_week_percent_accounting_wt= 20/100
ytc_sla_ltd_21_7_days_accounting_wt= 5/100
ytc_percent_ltd_21_7_days_accounting_wt= 5/100
#pip_percent_accounting_wt= 10/100
orders_completed_accounting_wt= 30/100
unique_seller_accounting_wt= 20/100
avg_feedback_accounting_wt= 20/100

In [285]:
## Accounting scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname'
       ]

accounting_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

accounting_rank=accounting_rank[accounting_rank['service_name']=='Accounting']
accounting_rank=accounting_rank.fillna(0)

##prepare the score

accounting_rank['score']=100*(  (ytc_percent_in_week_percent_accounting_wt*accounting_rank['ytc_in_week_percent']/accounting_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
                    + (ytc_sla_ltd_21_7_days_accounting_wt*accounting_rank['ytc_sla_21_7']/accounting_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
                    + (ytc_percent_ltd_21_7_days_accounting_wt*accounting_rank['ytc_ltd_21_7_percent']/accounting_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_accounting_wt*accounting_rank['pip_percent']
                    + (orders_completed_accounting_wt*accounting_rank['total_4w_orders']/accounting_rank['total_4w_orders'].max()).replace(np.nan,0)
                    + (unique_seller_accounting_wt*accounting_rank['adoption']/accounting_rank['adoption'].max()).replace(np.nan,0)
                    + (avg_feedback_accounting_wt*accounting_rank['avg_feedback']/accounting_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

accounting_rank['rank']=accounting_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname']
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
accounting_rank = accounting_rank.sort_values(by="rank", ignore_index=True)
accounting_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,c9f3c2fb-0628-485b-8308-4a03c8699434,Accounting,0.0,0.0,0.0,36.84,71.0,67.0,4.92,2022-12-09,2018-08-18,Jaipur Global Service(JGS),69.680000,1.0
1,43a1a46d-59be-41f9-b093-204afa9883cd,Accounting,100.0,0.0,0.0,100.00,4.0,1.0,5.00,2021-09-10,2021-09-10,NEELAM ECOMMERCE SOLUTION,41.988648,2.0
2,6af946b2-d8b9-4499-8005-90024d62404d,Accounting,0.0,0.0,0.0,50.00,21.0,20.0,5.00,2022-12-26,2022-01-05,ECOMLINK BUSINESS SOLUTIONS,34.843389,3.0
3,e3032be1-b2e1-41e9-8ee1-3346cb50b807,Accounting,0.0,0.0,0.0,0.00,21.0,12.0,5.00,2022-10-06,2020-03-26,$eller $olution,32.455329,4.0
4,825a388d-26d6-40a6-8e11-49e536698664,Accounting,100.0,0.0,100.0,0.00,0.0,0.0,1.00,2022-07-01,2021-06-07,Open.Money,29.000000,5.0


In [286]:
## let's have the weights for Training in this block

ytc_percent_in_week_percent_training_wt= 20/100
ytc_sla_ltd_21_7_days_training_wt= 5/100
ytc_percent_ltd_21_7_days_training_wt= 5/100
#pip_percent_training_wt= 10/100
orders_completed_training_wt= 30/100
unique_seller_training_wt= 20/100
avg_feedback_training_wt= 20/100

In [287]:
##Training scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

training_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

training_rank=training_rank[training_rank['service_name']=='Training']
training_rank=training_rank.fillna(0)

##prepare the score

training_rank['score']=100*(  (ytc_percent_in_week_percent_training_wt*training_rank['ytc_in_week_percent']/training_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
                    + (ytc_sla_ltd_21_7_days_training_wt*training_rank['ytc_sla_21_7']/training_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
                    + (ytc_percent_ltd_21_7_days_training_wt*training_rank['ytc_ltd_21_7_percent']/training_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_training_wt*training_rank['pip_percent']
                    + (orders_completed_training_wt*training_rank['total_4w_orders']/training_rank['total_4w_orders'].max()).replace(np.nan,0)
                    + (unique_seller_training_wt*training_rank['adoption']/training_rank['adoption'].max()).replace(np.nan,0)
                    + (avg_feedback_training_wt*training_rank['avg_feedback']/training_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

training_rank['rank']=training_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname']
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
training_rank = training_rank.sort_values(by="rank", ignore_index=True)
training_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Training,0.00,0.00,0.0,38.71,838.0,0.0,4.68,2023-02-15,2020-01-09,Professional Hub,48.720000,1.0
1,41332640-c25d-4f2b-bd52-38a9c2812cee,Training,0.00,0.00,0.0,50.00,65.0,3.0,5.00,2023-02-15,2018-10-26,Be Professional,42.326969,2.0
2,218919a7-661e-4e1c-a721-26be982ef3e7,Training,6.96,0.00,0.0,51.06,430.0,1.0,4.50,2021-12-30,2018-08-29,Calium Seller Services,41.452461,3.0
3,c9f3c2fb-0628-485b-8308-4a03c8699434,Training,0.00,0.00,0.0,25.00,119.0,2.0,5.00,2022-12-09,2018-08-18,Jaipur Global Service(JGS),37.593477,4.0
4,26ddd595-c3a2-413f-bec2-2de54264ede8,Training,10.60,5.74,0.0,46.43,382.0,0.0,5.00,2022-09-20,2018-08-28,Ecomm Enabler,36.082418,5.0


In [288]:
## let's have the weights for International Shipping in this block

ytc_percent_in_week_percent_i_s_wt= 20/100
ytc_sla_ltd_21_7_days_i_s_wt= 5/100
ytc_percent_ltd_21_7_days_i_s_wt= 5/100
#pip_percent_i_s_wt= 10/100
orders_completed_i_s_wt= 30/100
unique_seller_i_s_wt= 20/100
avg_feedback_i_s_wt= 20/100

In [289]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

i_s_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

i_s_rank=i_s_rank[i_s_rank['service_name']=='International Shipping']
i_s_rank=i_s_rank.fillna(0)

##prepare the score

i_s_rank['score']=100*(  (ytc_percent_in_week_percent_i_s_wt*i_s_rank['ytc_in_week_percent']/i_s_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
                    + (ytc_sla_ltd_21_7_days_i_s_wt*i_s_rank['ytc_sla_21_7']/i_s_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
                    + (ytc_percent_ltd_21_7_days_i_s_wt*i_s_rank['ytc_ltd_21_7_percent']/i_s_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_i_s_wt*i_s_rank['pip_percent']
                    + (orders_completed_i_s_wt*i_s_rank['total_4w_orders']/i_s_rank['total_4w_orders'].max()).replace(np.nan,0)
                    + (unique_seller_i_s_wt*i_s_rank['adoption']/i_s_rank['adoption'].max()).replace(np.nan,0)
                    + (avg_feedback_i_s_wt*i_s_rank['avg_feedback']/i_s_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

i_s_rank['rank']=i_s_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname']
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_s_rank = i_s_rank.sort_values(by="rank", ignore_index=True)
i_s_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,f28c3e53-bdb8-452c-8892-30c888af8991,International Shipping,100.0,75.0,25.0,40.0,0.0,0.0,0.0,2021-05-18,2021-01-22,FSL SHOP EASY DWC-LLC,25.0,1.0
1,b3b3e999-8cc7-4f13-b1ff-7bd4ce4a0648,International Shipping,100.0,0.0,100.0,0.0,0.0,0.0,0.0,2022-11-03,2020-06-22,RSA Global,25.0,2.0
2,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,International Shipping,100.0,0.0,100.0,0.0,0.0,0.0,0.0,2022-07-31,2019-12-27,Service Provider,25.0,3.0
3,44fa0c95-d4c5-4109-9a95-df0c028bd71b,International Shipping,100.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-19,2023-01-10,SF Express International,20.0,4.0
4,e74433a6-bf7c-43de-9545-84ea25ae8945,International Shipping,100.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-02,2022-05-03,ShipGlobal X,20.0,5.0


In [290]:
i_s_rank['total_4w_orders'].max()

0.0

In [291]:
## let's have the weights for Translation in this block

ytc_percent_in_week_percent_trans_wt= 20/100
ytc_sla_ltd_21_7_days_trans_wt= 5/100
ytc_percent_ltd_21_7_days_trans_wt= 5/100
#pip_percent_trans_wt= 10/100
orders_completed_trans_wt= 30/100
unique_seller_trans_wt= 20/100
avg_feedback_trans_wt= 20/100

In [292]:
##Translation scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

trans_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

trans_rank=trans_rank[trans_rank['service_name']=='Translation']
trans_rank=trans_rank.fillna(0)

##prepare the score

trans_rank['score']=100*(  
    (ytc_percent_in_week_percent_trans_wt*trans_rank['ytc_in_week_percent']/trans_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+ (ytc_sla_ltd_21_7_days_trans_wt*trans_rank['ytc_sla_21_7']/trans_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+ (ytc_percent_ltd_21_7_days_trans_wt*trans_rank['ytc_ltd_21_7_percent']/trans_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_i_s_wt*trans_rank['pip_percent']
+ (orders_completed_trans_wt*trans_rank['total_4w_orders']/trans_rank['total_4w_orders'].max()).replace(np.nan,0)
+ (unique_seller_trans_wt*trans_rank['adoption']/trans_rank['adoption'].max()).replace(np.nan,0)
+ (avg_feedback_trans_wt*trans_rank['avg_feedback']/trans_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

trans_rank['rank']=trans_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
trans_rank = trans_rank.sort_values(by="rank", ignore_index=True)
trans_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,71fcbc79-acd4-4ace-82cc-cd8ca682ecb5,Translation,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2022-08-08,2022-05-02,TheWonderKart,20.0,1.0
1,2d1a804d-29b5-4996-9a14-2cfe1312ac45,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-27,2023-01-18,Sellers Society,0.0,2.0
2,e0a39e44-9b16-43bb-8408-32cd96e8de8f,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-20,2022-11-10,ShopHowell,0.0,3.0
3,df33defb-2839-47d6-bea5-f51c050e428c,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-11,2021-04-07,PTSGI（統一翻訳株式会社）,0.0,4.0
4,0fe95224-9a76-4672-9624-30b8f0509d31,Translation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-05,2018-08-27,eStore Factory,0.0,5.0


In [293]:
## let's have the weights for International Returns in this block

ytc_percent_in_week_percent_i_r_wt= 20/100
ytc_sla_ltd_21_7_days_i_r_wt= 5/100
ytc_percent_ltd_21_7_days_i_r_wt= 5/100
#pip_percent_i_r_wt= 10/100
orders_completed_i_r_wt= 30/100
unique_seller_i_r_wt= 20/100
avg_feedback_i_r_wt= 20/100

In [294]:
##International Shipping scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

i_r_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

i_r_rank=i_r_rank[i_r_rank['service_name']=='International Returns']
i_r_rank=i_r_rank.fillna(0)

##prepare the score

i_r_rank['score']=100*(  
    (ytc_percent_in_week_percent_i_r_wt*i_r_rank['ytc_in_week_percent']/i_r_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+ (ytc_sla_ltd_21_7_days_i_r_wt*i_r_rank['ytc_sla_21_7']/i_r_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+ (ytc_percent_ltd_21_7_days_i_r_wt*i_r_rank['ytc_ltd_21_7_percent']/i_r_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_i_r_wt*i_r_rank['pip_percent']
+ (orders_completed_i_r_wt*i_r_rank['total_4w_orders']/i_r_rank['total_4w_orders'].max()).replace(np.nan,0)
+ (unique_seller_i_r_wt*i_r_rank['adoption']/i_r_rank['adoption'].max()).replace(np.nan,0)
+ (avg_feedback_i_r_wt*i_r_rank['avg_feedback']/i_r_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

i_r_rank['rank']=i_r_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
i_r_rank = i_r_rank.sort_values(by="rank", ignore_index=True)
i_r_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,f28c3e53-bdb8-452c-8892-30c888af8991,International Returns,100.0,0.0,100.0,0.0,0.0,0.0,0.0,2021-05-18,2021-01-22,FSL SHOP EASY DWC-LLC,25.0,1.0
1,4e30b5b2-7df3-482e-847e-a556bdbdba46,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-22,2023-02-22,EPS EXPRESS,0.0,2.0
2,a88a928d-e353-478d-a409-1c9910332c74,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-22,2022-10-12,"Shanghai Jieli Transportation Co., Ltd.",0.0,3.0
3,5262aae5-1fbc-4056-999f-028fd22ab7a9,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-22,2019-06-18,"Shenzhen Baosen Suntop Logistics Co.,Ltd",0.0,4.0
4,bf65248b-1a79-47f2-84c4-9f1bdf44ecfd,International Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-15,2023-02-15,YunExpress,0.0,5.0


In [295]:
## let's have the weights for FBA in this block

ytc_percent_in_week_percent_fba_wt= 20/100
ytc_sla_ltd_21_7_days_fba_wt= 5/100
ytc_percent_ltd_21_7_days_fba_wt= 5/100
#pip_percent_fba_wt= 10/100
orders_completed_fba_wt= 30/100
unique_seller_fba_wt= 20/100
avg_feedback_fba_wt= 20/100

In [296]:
## FBA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

fba_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

fba_rank=fba_rank[fba_rank['service_name']=='FBA Preparation']
fba_rank=fba_rank.fillna(0)

##prepare the score

fba_rank['score']=100*(  
    (ytc_percent_in_week_percent_fba_wt*fba_rank['ytc_in_week_percent']/fba_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+ (ytc_sla_ltd_21_7_days_fba_wt*fba_rank['ytc_sla_21_7']/fba_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+ (ytc_percent_ltd_21_7_days_fba_wt*fba_rank['ytc_ltd_21_7_percent']/fba_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_fba_wt*fba_rank['pip_percent']
+ (orders_completed_fba_wt*fba_rank['total_4w_orders']/fba_rank['total_4w_orders'].max()).replace(np.nan,0)
+ (unique_seller_fba_wt*fba_rank['adoption']/fba_rank['adoption'].max()).replace(np.nan,0)
+ (avg_feedback_fba_wt*fba_rank['avg_feedback']/fba_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

fba_rank['rank']=fba_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname']
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
fba_rank = fba_rank.sort_values(by="rank", ignore_index=True)
fba_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,504878ee-ff2f-46b3-98f7-77d5e7ed139b,FBA Preparation,0.0,0.0,0.0,22.22,47.0,25.0,5.0,2022-06-03,2021-01-19,Ecom Multi Service,70.000000,1.0
1,9024e1be-44b7-4d2e-9d2d-6db10e8c0ded,FBA Preparation,0.0,0.0,0.0,0.00,14.0,14.0,5.0,2023-02-27,2022-12-20,BELIEVERS,40.136170,2.0
2,d2f317ff-cbd0-4846-9b47-18fe8273c6ae,FBA Preparation,0.0,0.0,0.0,0.00,18.0,10.0,5.0,2022-09-22,2021-10-04,Seller Service Provider,39.489362,3.0
3,a021786e-3c21-48be-955a-549382f3a3ae,FBA Preparation,0.0,0.0,0.0,0.00,13.0,13.0,5.0,2023-01-19,2018-08-26,Loombiz Consultancy,38.697872,4.0
4,b8e4a5b0-a3df-4d60-8394-f1dc81bcad6c,FBA Preparation,0.0,0.0,0.0,70.00,2.0,2.0,5.0,2022-12-19,2019-05-07,FLICKERING CONSULTANCY PVT LTD,22.876596,5.0


In [297]:
## let's have the weights for storage in this block

ytc_percent_in_week_percent_stor_wt= 20/100
ytc_sla_ltd_21_7_days_stor_wt= 5/100
ytc_percent_ltd_21_7_days_stor_wt= 5/100
#pip_percent_stor_wt= 10/100
orders_completed_stor_wt= 30/100
unique_seller_stor_wt= 20/100
avg_feedback_stor_wt= 20/100

In [298]:
## Storage scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

stor_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

stor_rank=stor_rank[stor_rank['service_name']=='Storage']
stor_rank=stor_rank.fillna(0)

##prepare the score

stor_rank['score']=100*(  
    (ytc_percent_in_week_percent_stor_wt*stor_rank['ytc_in_week_percent']/stor_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
  + (ytc_sla_ltd_21_7_days_stor_wt*stor_rank['ytc_sla_21_7']/stor_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
  + (ytc_percent_ltd_21_7_days_stor_wt*stor_rank['ytc_ltd_21_7_percent']/stor_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_stor_wt*stor_rank['pip_percent']
  + (orders_completed_stor_wt*stor_rank['total_4w_orders']/stor_rank['total_4w_orders'].max()).replace(np.nan,0)
  + (unique_seller_stor_wt*stor_rank['adoption']/stor_rank['adoption'].max()).replace(np.nan,0)
  + (avg_feedback_stor_wt*stor_rank['avg_feedback']/stor_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

stor_rank['rank']=stor_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
stor_rank = stor_rank.sort_values(by="rank", ignore_index=True)
stor_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,ad3442f1-f0c5-4a84-b3c3-64a61fe4dbe7,Storage,10.34,0.00,0.00,0.00,120.0,111.0,5.0,2022-04-29,2020-02-18,The Bitti Enterprises,72.068000,1.0
1,bbef9f87-0261-4212-9b6e-60922e19c662,Storage,100.00,7.14,35.71,0.00,39.0,29.0,5.0,2021-09-28,2021-09-28,Adifound,57.117725,2.0
2,fb45d4ec-2467-4035-a153-93ec5331d753,Storage,2.27,0.00,0.00,0.00,110.0,107.0,1.0,2022-02-04,2020-01-31,Seller Café,51.233279,3.0
3,c9f3c2fb-0628-485b-8308-4a03c8699434,Storage,0.00,0.00,0.00,64.29,63.0,63.0,5.0,2022-12-09,2018-08-18,Jaipur Global Service(JGS),47.101351,4.0
4,c8e28a15-bfc7-44cf-bbcc-2825567ffede,Storage,0.00,66.67,0.00,0.00,6.0,5.0,5.0,2022-04-27,2021-06-09,Seva sales Corporation,25.734401,5.0


In [299]:
## let's have the weights for IPA in this block

ytc_percent_in_week_percent_ipa_wt= 20/100
ytc_sla_ltd_21_7_days_ipa_wt= 5/100
ytc_percent_ltd_21_7_days_ipa_wt= 5/100
#pip_percent_ipa_wt= 10/100
orders_completed_ipa_wt= 30/100
unique_seller_ipa_wt= 20/100
avg_feedback_ipa_wt= 20/100

In [300]:
## IPA scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

ipa_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

ipa_rank=ipa_rank[ipa_rank['service_name']=='IP Accelerator']
ipa_rank=ipa_rank.fillna(0)

##prepare the score

ipa_rank['score']=100*(  
    (ytc_percent_in_week_percent_ipa_wt*ipa_rank['ytc_in_week_percent']/ipa_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+   (ytc_sla_ltd_21_7_days_ipa_wt*ipa_rank['ytc_sla_21_7']/ipa_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+   (ytc_percent_ltd_21_7_days_ipa_wt*ipa_rank['ytc_ltd_21_7_percent']/ipa_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_ipa_wt*ipa_rank['pip_percent']
+   (orders_completed_ipa_wt*ipa_rank['total_4w_orders']/ipa_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_ipa_wt*ipa_rank['adoption']/ipa_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_ipa_wt*ipa_rank['avg_feedback']/ipa_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

ipa_rank['rank']=ipa_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ipa_rank = ipa_rank.sort_values(by="rank", ignore_index=True)
ipa_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,IP Accelerator,100.00,0.00,100.0,0.00,0.0,0.0,0.0,2022-07-31,2019-12-27,Service Provider,25.0000,1.0
1,bef07376-64e0-4df9-b5c0-8ba8c135ea04,IP Accelerator,100.00,40.00,40.0,80.00,0.0,0.0,0.0,2021-07-28,2021-07-28,R.K. Dewan & Co.,24.0000,2.0
2,fe763b02-f44d-4129-810a-3020d7afde04,IP Accelerator,8.96,3.61,0.0,12.05,0.0,0.0,5.0,2021-11-18,2020-12-02,LexOrbis,21.9725,3.0
3,70729788-8930-43c2-aff6-3f33fdc921d3,IP Accelerator,3.57,0.00,2.0,54.55,0.0,0.0,5.0,2021-06-07,2021-06-07,Amitabha Sen & co.,20.8140,4.0
4,685b69a4-690a-4e60-b312-da59f31c46a0,IP Accelerator,50.00,66.67,0.0,0.00,0.0,0.0,0.0,2021-06-22,2021-06-22,Remfry & Sagar,13.3335,5.0


In [301]:
## let's have the weights for Excess Inventory  in this block

ytc_percent_in_week_percent_exec_wt= 20/100
ytc_sla_ltd_21_7_days_exec_wt= 5/100
ytc_percent_ltd_21_7_days_exec_wt= 5/100
#pip_percent_exec_wt= 10/100
orders_completed_exec_wt= 30/100
unique_seller_exec_wt= 20/100
avg_feedback_exec_wt= 20/100

In [302]:
## Excesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'updated_date',
        'onboarding_date',
        'displayname']

exec_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

exec_rank=exec_rank[exec_rank['service_name']=='Excess Inventory']
exec_rank=exec_rank.fillna(0)

##prepare the score

exec_rank['score']=100*(  
    (ytc_percent_in_week_percent_exec_wt*exec_rank['ytc_in_week_percent']/exec_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+   (ytc_sla_ltd_21_7_days_exec_wt*exec_rank['ytc_sla_21_7']/exec_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+   (ytc_percent_ltd_21_7_days_exec_wt*exec_rank['ytc_ltd_21_7_percent']/exec_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_exec_wt*exec_rank['pip_percent']
+   (orders_completed_exec_wt*exec_rank['total_4w_orders']/exec_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_exec_wt*exec_rank['adoption']/exec_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_exec_wt*exec_rank['avg_feedback']/exec_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

exec_rank['rank']=exec_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'updated_date',
     'onboarding_date',
     'displayname']].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
exec_rank = exec_rank.sort_values(by="rank", ignore_index=True)
exec_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,updated_date,onboarding_date,displayname,score,rank
0,b6209911-111c-448f-b152-4d1ccd8a91b9,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,2021-12-16,2021-12-16,ELECTRONIC HUT,25.0000,1.0
1,0a8e300b-68a1-42b5-9ad1-c953710dde00,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,2021-12-14,2021-12-14,VGM Traders,25.0000,2.0
2,5a3bdad8-0f7b-45be-bade-d2e8b13d468f,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,2021-12-14,2021-12-14,SURPLUS WORLD,25.0000,3.0
3,bc3c2364-16f3-43b9-b775-17a921b27459,Excess Inventory,100.0,0.0,100.00,0.0,0.0,0.0,0.0,2021-12-02,2021-12-02,Deepak Enterprises,25.0000,4.0
4,c0d52ba3-3eb7-457e-a8f0-02bfe947ea24,Excess Inventory,100.0,0.0,66.67,100.0,0.0,0.0,0.0,2021-12-02,2021-10-28,Dikksh,23.3335,5.0


In [303]:
## let's have the weights for Cataloging in this block

ytc_percent_in_week_percent_cat_wt= 10/100
ytc_sla_ltd_21_7_days_cat_wt= 10/100
ytc_percent_ltd_21_7_days_cat_wt= 10/100
#pip_percent_cat_wt= 10/100
orders_completed_cat_wt= 20/100
unique_seller_cat_wt= 10/100
avg_feedback_cat_wt= 10/100
caq_cat_wt=20/100
cat_asin_created_wt=10/100

In [304]:
## Excesss Catalouging scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'caq_score',
        'count_of_asins',
        'updated_date',
        'onboarding_date',
        'displayname']

cat_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

cat_rank=cat_rank[cat_rank['service_name']=='Cataloguing']
cat_rank=cat_rank.fillna(0)

##prepare the score

cat_rank['score']=100*(  
    (ytc_percent_in_week_percent_cat_wt*cat_rank['ytc_in_week_percent']/cat_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+   (ytc_sla_ltd_21_7_days_cat_wt*cat_rank['ytc_sla_21_7']/cat_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+   (ytc_percent_ltd_21_7_days_cat_wt*cat_rank['ytc_ltd_21_7_percent']/cat_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_cat_wt*cat_rank['pip_percent']
+   (orders_completed_cat_wt*cat_rank['total_4w_orders']/cat_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_cat_wt*cat_rank['adoption']/cat_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_cat_wt*cat_rank['avg_feedback']/cat_rank['avg_feedback'].max()).replace(np.nan,0)
+   (caq_cat_wt*cat_rank['caq_score']/cat_rank['caq_score'].max())
+   (cat_asin_created_wt*cat_rank['count_of_asins']/cat_rank['count_of_asins'].max()).replace(np.nan,0)
                   )

cat_rank['rank']=cat_rank[
    ['score',
     'total_4w_orders',
     'caq_score',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'count_of_asins',
     'updated_date',
     'onboarding_date',
     'displayname'


]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
cat_rank = cat_rank.sort_values(by="rank", ignore_index=True)
cat_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,caq_score,count_of_asins,updated_date,onboarding_date,displayname,score,rank
0,b3f2f78a-5443-48ea-8b17-28b337100ccc,Cataloguing,0.00,0.49,0.0,72.40,682.0,0.0,5.00,0.74,1685091.0,2022-10-06,2018-08-16,REEFORMERS,52.725802,1.0
1,26ddd595-c3a2-413f-bec2-2de54264ede8,Cataloguing,0.12,0.00,0.0,46.63,763.0,0.0,4.12,0.82,42580.0,2022-09-20,2018-08-28,Ecomm Enabler,44.904687,2.0
2,860468c6-9e94-4627-8e89-23e58e944fff,Cataloguing,0.00,0.00,0.0,42.61,504.0,0.0,4.81,0.83,196000.0,2023-01-05,2018-08-27,e-Vitamins,40.594151,3.0
3,7efb2627-714b-45d2-b7cb-9308ad5ef017,Cataloguing,0.00,100.00,0.0,100.00,2.0,0.0,5.00,1.00,21.0,2022-06-01,2018-08-22,Ecom services 24X7,40.052549,4.0
4,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Cataloguing,0.28,1.92,0.0,30.97,462.0,0.0,4.47,0.79,346212.0,2021-08-12,2018-10-08,Value For Sellers,39.124651,5.0


In [305]:
## let's have the weights for taxes in this block

ytc_percent_in_week_percent_tax_wt= 20/100
ytc_sla_ltd_21_7_days_tax_wt= 5/100
ytc_percent_ltd_21_7_days_tax_wt= 5/100
#pip_percent_tax_wt= 10/100
orders_completed_tax_wt= 30/100
unique_seller_tax_wt= 20/100
avg_feedback_tax_wt= 20/100

In [306]:
## Taxesss Inventory scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'tax_unblocked_gms',
        'updated_date',
        'onboarding_date',
        'displayname']

tax_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

tax_rank=tax_rank[tax_rank['service_name']=='Taxes']
tax_rank=tax_rank.fillna(0)

##prepare the score

tax_rank['score']=100*(  
    (ytc_percent_in_week_percent_tax_wt*tax_rank['ytc_in_week_percent']/tax_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+   (ytc_sla_ltd_21_7_days_tax_wt*tax_rank['ytc_sla_21_7']/tax_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+   (ytc_percent_ltd_21_7_days_tax_wt*tax_rank['ytc_ltd_21_7_percent']/tax_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_tax_wt*tax_rank['pip_percent']
+  (orders_completed_tax_wt*tax_rank['total_4w_orders']/tax_rank['total_4w_orders'].max()).replace(np.nan,0)
+  (unique_seller_tax_wt*tax_rank['adoption']/tax_rank['adoption'].max()).replace(np.nan,0)
+  (avg_feedback_tax_wt*tax_rank['avg_feedback']/tax_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

tax_rank['rank']=tax_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
    'ytc_sla_21_7',
    'ytc_ltd_21_7_percent',
    'tax_unblocked_gms',
    'updated_date',
     'onboarding_date',
     'displayname']
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
tax_rank = tax_rank.sort_values(by="rank", ignore_index=True)
tax_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,tax_unblocked_gms,updated_date,onboarding_date,displayname,score,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Taxes,0.00,0.0,0.0,62.50,732.0,530.0,5.0,1.630258e+06,2021-08-12,2018-10-08,Value For Sellers,70.000000,1.0
1,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Taxes,1.05,0.0,0.0,43.75,430.0,424.0,0.0,5.768702e+05,2022-10-14,2019-04-02,Lynex Global Solutions,33.832951,2.0
2,c9f3c2fb-0628-485b-8308-4a03c8699434,Taxes,0.00,0.0,0.0,64.29,114.0,58.0,5.0,3.078227e+08,2022-12-09,2018-08-18,Jaipur Global Service(JGS),26.860810,3.0
3,edc4f8ac-e14d-4e43-af61-399cabbbacf7,Taxes,0.00,100.0,0.0,0.00,4.0,4.0,5.0,3.229127e+06,2022-07-25,2022-07-20,The GST Company,25.314878,4.0
4,de23db03-abe4-4652-ab19-1f67eb52079c,Taxes,100.00,100.0,0.0,100.00,0.0,0.0,0.0,0.000000e+00,2023-02-15,2020-07-31,Lucria Consult,25.000000,5.0


In [307]:
## let's have the weights for compliance in this block

ytc_percent_in_week_percent_comp_wt= 20/100
ytc_sla_ltd_21_7_days_comp_wt= 5/100
ytc_percent_ltd_21_7_days_comp_wt= 5/100
#pip_percent_comp_wt= 10/100
orders_completed_comp_wt= 30/100
unique_seller_comp_wt= 20/100
avg_feedback_comp_wt= 20/100

In [308]:
## Excesss Compliance scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_percent',
        'ytc_sla_21_7',
        'ytc_ltd_21_7_percent',
        'pip_percent',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'compliance_unblocked_gms',
        'unblocked_asin',
        'updated_date',
        'onboarding_date',
        'displayname']

comp_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

comp_rank=comp_rank[comp_rank['service_name']=='Compliance']
comp_rank=comp_rank.fillna(0)

##prepare the score

comp_rank['score']=100*(  
    (ytc_percent_in_week_percent_comp_wt*comp_rank['ytc_in_week_percent']/comp_rank['ytc_in_week_percent'].max()).replace(np.nan,0)
+   (ytc_sla_ltd_21_7_days_comp_wt*comp_rank['ytc_sla_21_7']/comp_rank['ytc_sla_21_7'].max()).replace(np.nan,0)
+   (ytc_percent_ltd_21_7_days_comp_wt*comp_rank['ytc_ltd_21_7_percent']/comp_rank['ytc_ltd_21_7_percent'].max()).replace(np.nan,0)
                    #+ pip_percent_comp_wt*comp_rank['pip_percent']
+   (orders_completed_comp_wt*comp_rank['total_4w_orders']/comp_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_comp_wt*comp_rank['adoption']/comp_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_comp_wt*comp_rank['avg_feedback']/comp_rank['avg_feedback'].max()).replace(np.nan,0)
                   )

comp_rank['rank']=comp_rank[
    ['score',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'ytc_in_week_percent',
     'ytc_sla_21_7',
     'ytc_ltd_21_7_percent',
     'compliance_unblocked_gms',
     'unblocked_asin',
     'updated_date',
     'onboarding_date',
     'displayname'
    ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
comp_rank = comp_rank.sort_values(by="rank", ignore_index=True)
comp_rank.head()

,provider_id,service_name,ytc_in_week_percent,ytc_sla_21_7,ytc_ltd_21_7_percent,pip_percent,total_4w_orders,adoption,avg_feedback,compliance_unblocked_gms,unblocked_asin,updated_date,onboarding_date,displayname,score,rank
0,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Compliance,0.0,0.0,0.0,36.36,196.0,83.0,0.0,0.00,0.0,2022-10-14,2019-04-02,Lynex Global Solutions,50.000000,1.0
1,8ab05dea-5450-42ed-b06a-b3e49a8e1f05,Compliance,0.0,0.0,0.0,0.00,139.0,56.0,0.0,221645.97,24.0,2022-09-22,2022-09-22,VIRTUAL BUDDIES,34.769486,2.0
2,6a9c04cb-4c34-4fa6-8d4a-b65561aeec1c,Compliance,100.0,0.0,100.0,0.00,0.0,0.0,0.0,0.00,0.0,2022-07-31,2019-12-27,Service Provider,25.000000,3.0
3,00bb70b7-b526-430d-8979-584b329b6746,Compliance,100.0,0.0,100.0,0.00,0.0,0.0,0.0,0.00,0.0,2021-09-17,2021-09-17,TUV INDIA PRIVATE LIMITED,25.000000,4.0
4,6340a578-2a34-4e9f-800a-ffb14e6ac0ac,Compliance,100.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,2021-02-19,2020-06-26,Mevada & Co,20.000000,5.0


In [309]:
## for adz account management ebc and DS, we have ytc my understanding is it is ytc to contact numerator.this we have to check 

In [310]:
## let's have the weights for adz in this block

ytc_percent_in_week_num_adz_wt= 20/100
orders_completed_adz_wt= 20/100
unique_seller_adz_wt= 15/100
avg_feedback_adz_wt= 15/100
acos_adz_wt=10/100
ad_revenue_adz_wt=25/100

In [311]:
## Ad Optimization scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'total_spend',
        'acos_p',
        'updated_date',
        'onboarding_date',
        'displayname']

adz_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

adz_rank=adz_rank[adz_rank['service_name']=='Advertising Optimization']
adz_rank=adz_rank.fillna(0)

##prepare the score

adz_rank['score']=100*( 
    (ytc_percent_in_week_num_adz_wt*adz_rank['ytc_in_week_numerator']/adz_rank['ytc_in_week_numerator'].max()).replace(np.nan,0)
+   (orders_completed_adz_wt*adz_rank['total_4w_orders']/adz_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_adz_wt*adz_rank['adoption']/adz_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_adz_wt*adz_rank['avg_feedback']/adz_rank['avg_feedback'].max()).replace(np.nan,0)
+   (ad_revenue_adz_wt*adz_rank['total_spend']/adz_rank['total_spend'].max()).replace(np.nan,0)
+   (acos_adz_wt*adz_rank['acos_p']/adz_rank['acos_p'].max()).replace(np.nan,0)
                   )
adz_rank['rank']=adz_rank[
    ['score',
     'total_spend',
     'ytc_in_week_numerator',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'acos_p',
     'updated_date',
     'onboarding_date',
     'displayname'
     ]].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
adz_rank = adz_rank.sort_values(by="rank", ignore_index=True)
adz_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,total_spend,acos_p,updated_date,onboarding_date,displayname,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Advertising Optimization,0.0,600.0,31.0,4.56,1.157713e+08,0.217226,2023-01-05,2018-08-27,e-Vitamins,74.518541,1.0
1,504878ee-ff2f-46b3-98f7-77d5e7ed139b,Advertising Optimization,7.0,1.0,0.0,5.00,1.742071e+04,0.087780,2022-06-03,2021-01-19,Ecom Multi Service,35.375948,2.0
2,c9f3c2fb-0628-485b-8308-4a03c8699434,Advertising Optimization,0.0,288.0,0.0,4.91,2.082623e+07,0.214337,2022-12-09,2018-08-18,Jaipur Global Service(JGS),29.654664,3.0
3,ac1398cb-72ca-443c-8ae6-8f764e1297f4,Advertising Optimization,0.0,3.0,0.0,5.00,5.859970e+07,0.244733,2022-11-08,2018-08-22,BM Consulting,28.698912,4.0
4,8efd01b7-d87b-4ba9-acf1-172256164c85,Advertising Optimization,0.0,6.0,0.0,5.00,5.112100e+07,0.411462,2021-09-03,2019-03-19,Seller Circle,27.827555,5.0


In [312]:
## let's have the weights for account management in this block

ytc_percent_in_week_num_am_wt= 20/100
orders_completed_am_wt= 20/100
unique_seller_am_wt= 15/100
avg_feedback_am_wt= 15/100
am_gms_wt=30/100

In [313]:
## AM scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'boost_gms',
        'updated_date',
        'onboarding_date',
        'displayname']

am_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

am_rank=am_rank[am_rank['service_name']=='Account Management']
am_rank=am_rank.fillna(0)

##prepare the score

am_rank['score']=100*( 
    (ytc_percent_in_week_num_am_wt*am_rank['ytc_in_week_numerator']/am_rank['ytc_in_week_numerator'].max()).replace(np.nan,0)
+   (orders_completed_am_wt*am_rank['total_4w_orders']/am_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_am_wt*am_rank['adoption']/am_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_am_wt*am_rank['avg_feedback']/am_rank['avg_feedback'].max()).replace(np.nan,0)
+   (am_gms_wt*am_rank['boost_gms']/am_rank['boost_gms'].max()).replace(np.nan,0)
                   )

am_rank['rank']=am_rank[
    ['score',
     'boost_gms',
     'ytc_in_week_numerator',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'updated_date',
     'onboarding_date',
     'displayname'
     ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
am_rank = am_rank.sort_values(by="rank", ignore_index=True)
am_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,boost_gms,updated_date,onboarding_date,displayname,score,rank
0,b975ba43-4856-42de-8857-7d3551e647b3,Account Management,1.0,100.0,0.0,4.55,7.614578e+08,2022-12-05,2020-01-06,Newgentech Solution,57.499558,1.0
1,860468c6-9e94-4627-8e89-23e58e944fff,Account Management,3.0,226.0,4.0,4.79,7.281577e+07,2023-01-05,2018-08-27,e-Vitamins,57.238804,2.0
2,04407379-d775-4a69-9d63-bf5b5e01b282,Account Management,2.0,109.0,1.0,4.96,1.363627e+08,2021-03-25,2018-08-17,Big faction consultancy pvt ltd,41.148451,3.0
3,0611cfd2-2c7f-44e1-9a2c-e6f60cdca5cd,Account Management,4.0,20.0,0.0,5.00,8.808113e+05,2022-06-02,2021-05-20,TRIO INDIA,36.804614,4.0
4,2a64ecda-082d-4361-b5c4-2cad8740c59d,Account Management,0.0,43.0,12.0,5.00,4.355710e+07,2022-09-27,2018-09-21,SSP E-Commerce Services Pvt Ltd,35.521377,5.0


In [314]:
## let's have the weights for EBC in this block

ytc_percent_in_week_num_ebc_wt= 15/100
orders_completed_ebc_wt= 20/100
unique_seller_ebc_wt= 15/100
avg_feedback_ebc_wt= 15/100
ebc_gv_wt=25/100
ebc_asin_coverage_wt=10/100

In [315]:
## EBC scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ebc_gv_asins_created',
        'lw_gv_platform',
        'updated_date',
        'onboarding_date',
        'displayname']

ebc_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

ebc_rank=ebc_rank[ebc_rank['service_name']=='Enhanced Brand Content']
ebc_rank=ebc_rank.fillna(0)

##prepare the score

ebc_rank['score']=100*( 
    (ytc_percent_in_week_num_ebc_wt*ebc_rank['ytc_in_week_numerator']/ebc_rank['ytc_in_week_numerator'].max()).replace(np.nan,0)
+   (orders_completed_ebc_wt*ebc_rank['total_4w_orders']/ebc_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_ebc_wt*ebc_rank['adoption']/ebc_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_ebc_wt*ebc_rank['avg_feedback']/ebc_rank['avg_feedback'].max()).replace(np.nan,0)
+   (ebc_gv_wt*ebc_rank['lw_gv_platform']/ebc_rank['lw_gv_platform'].max()).replace(np.nan,0)
+   (ebc_asin_coverage_wt*ebc_rank['ebc_gv_asins_created']/ebc_rank['ebc_gv_asins_created'].max()).replace(np.nan,0)
                   )

ebc_rank['rank']=ebc_rank[
    ['score',
     'lw_gv_platform',
     'total_4w_orders',
     'ytc_in_week_numerator',
     'avg_feedback',
     'adoption',
     'ebc_gv_asins_created',
     'updated_date',
     'onboarding_date',
     'displayname'
     ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ebc_rank = ebc_rank.sort_values(by="rank", ignore_index=True)
ebc_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ebc_gv_asins_created,lw_gv_platform,updated_date,onboarding_date,displayname,score,rank
0,860468c6-9e94-4627-8e89-23e58e944fff,Enhanced Brand Content,0.0,0.0,0.0,4.48,17169.0,2330284.0,2023-01-05,2018-08-27,e-Vitamins,48.440000,1.0
1,a3edbea2-0b75-494c-80b9-d8fa529f2b31,Enhanced Brand Content,1.0,0.0,0.0,5.00,15260.0,2147188.0,2022-07-11,2018-09-03,DIGITAL CONCEPTUM,48.423802,2.0
2,f6ab326d-bcd5-4f98-bd54-3c058782f049,Enhanced Brand Content,0.0,0.0,0.0,4.75,2975.0,1786724.0,2020-06-16,2018-08-26,UGLY,35.151296,3.0
3,26ddd595-c3a2-413f-bec2-2de54264ede8,Enhanced Brand Content,1.0,0.0,0.0,5.00,3044.0,394192.0,2022-09-20,2018-08-28,Ecomm Enabler,22.501975,4.0
4,e624d8bb-038f-4e19-97bf-0f949769944b,Enhanced Brand Content,0.0,0.0,0.0,0.00,4158.0,1825205.0,2020-03-31,2018-08-22,Tech2Globe,22.003165,5.0


In [316]:
## let's have the weights for SNR in this block

ytc_percent_in_week_num_snr_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_snr_wt= 20/100
unique_seller_snr_wt= 15/100
avg_feedback_snr_wt= 15/100
snr_reinstatement_p_wt=30/100

In [317]:
## SNR scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'snr_reinst_p',
        'updated_date',
        'onboarding_date',
        'displayname']

snr_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

snr_rank=snr_rank[snr_rank['service_name']=='Seller Reinstatement']
snr_rank=snr_rank.fillna(0)

##prepare the score

snr_rank['score']=100*( 
    (ytc_percent_in_week_num_snr_wt*snr_rank['ytc_in_week_numerator']/snr_rank['ytc_in_week_numerator'].max()).replace(np.nan,0)
+   (orders_completed_snr_wt*snr_rank['total_4w_orders']/snr_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_snr_wt*snr_rank['adoption']/snr_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_snr_wt*snr_rank['avg_feedback']/snr_rank['avg_feedback'].max()).replace(np.nan,0)
+   (snr_reinstatement_p_wt*snr_rank['snr_reinst_p']/snr_rank['snr_reinst_p'].max()).replace(np.nan,0)
                   )

snr_rank['rank']=snr_rank[
    ['score',
     'snr_reinst_p',
     'ytc_in_week_numerator',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'updated_date',
     'onboarding_date',
     'displayname'
     ]
].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
snr_rank = snr_rank.sort_values(by="rank", ignore_index=True)
snr_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,snr_reinst_p,updated_date,onboarding_date,displayname,score,rank
0,90fb720f-19d1-4bab-b1ab-c918355b7b4d,Seller Reinstatement,1.0,800.0,791.0,3.66,0.0151,2022-06-30,2018-08-28,Rapra,41.558347,1.0
1,e4c1a0fb-82d1-4286-9774-26daa40a73e5,Seller Reinstatement,6.0,7.0,7.0,5.00,0.0870,2022-05-18,2022-05-18,Kaushal E-Com Solution,36.542575,2.0
2,88c9ed93-d745-47db-95b7-1cc8acb1192b,Seller Reinstatement,0.0,1043.0,1016.0,0.00,0.0017,2020-07-31,2018-08-18,Ace e-Commerce Solutions,35.025500,3.0
3,61504202-50d7-4ead-9046-60471bcd304f,Seller Reinstatement,0.0,1.0,1.0,0.00,2.0000,2021-06-17,2018-08-16,Ashish Pandey,30.033939,4.0
4,dbb45b0d-7f9b-4a12-8b28-df95238f1ebc,Seller Reinstatement,0.0,0.0,0.0,5.00,1.0000,2022-01-17,2021-07-08,VGC E- COMMERCE SOLUTION,30.000000,5.0


In [318]:
## let's have the weights for DS in this block

ytc_percent_in_week_num_ds_wt= 20/100
#pip_percent_snr_wt= 10/100
orders_completed_ds_wt= 20/100
unique_seller_ds_wt= 15/100
avg_feedback_ds_wt= 15/100
ds_ncns_p_wt=30/100

In [319]:
## DS scorecard

## take all the needed columns for the services
colsss=['provider_id',
        'service_name',
        'ytc_in_week_numerator',
        'total_4w_orders',
        'adoption',
        'avg_feedback',
        'ncns',
        'updated_date',
        'onboarding_date',
        'displayname']

ds_rank = join_1_21[colsss]

## filter for the service and replace all the blanks with NAN values

ds_rank=ds_rank[ds_rank['service_name']=='Domestic Shipping']
ds_rank=ds_rank.fillna(0)

##prepare the score

ds_rank['score']=100*( 
    (ytc_percent_in_week_num_ds_wt*ds_rank['ytc_in_week_numerator']/ds_rank['ytc_in_week_numerator'].max()).replace(np.nan,0)
+   (orders_completed_ds_wt*ds_rank['total_4w_orders']/ds_rank['total_4w_orders'].max()).replace(np.nan,0)
+   (unique_seller_ds_wt*ds_rank['adoption']/ds_rank['adoption'].max()).replace(np.nan,0)
+   (avg_feedback_ds_wt*ds_rank['avg_feedback']/ds_rank['avg_feedback'].max()).replace(np.nan,0)
+   (ds_ncns_p_wt*ds_rank['ncns']/ds_rank['ncns'].max()).replace(np.nan,0)
                   )

ds_rank['rank']=ds_rank[
    ['score',
     'ncns',
     'ytc_in_week_numerator',
     'total_4w_orders',
     'avg_feedback',
     'adoption',
     'updated_date',
     'onboarding_date',
     'displayname'
     ]].apply(
                    tuple, axis=1
                    ).rank(method='dense',ascending=False)# sort values
ds_rank = ds_rank.sort_values(by="rank", ignore_index=True)
ds_rank.head()

,provider_id,service_name,ytc_in_week_numerator,total_4w_orders,adoption,avg_feedback,ncns,updated_date,onboarding_date,displayname,score,rank
0,17ab417b-f0fb-4e42-becd-b53ce59ac0b6,Domestic Shipping,1.0,0.0,0.0,0.0,0.0195,2021-08-19,2018-09-03,S2 LOGISTICS,31.666667,1.0
1,186b1aa4-24a8-454a-8aea-d4517546408e,Domestic Shipping,12.0,0.0,0.0,0.0,0.0000,2020-04-20,2018-11-30,Spoton Logistics Private Limited,20.000000,2.0
2,71fcbc79-acd4-4ace-82cc-cd8ca682ecb5,Domestic Shipping,0.0,0.0,0.0,4.0,0.0000,2022-08-08,2022-05-02,TheWonderKart,15.000000,3.0
3,9aec9e35-135b-417e-af34-ce19772b8edd,Domestic Shipping,8.0,0.0,0.0,0.0,0.0000,2020-01-10,2018-09-03,Safexpress Pvt Ltd,13.333333,4.0
4,9a6ef1c7-6e8b-4e90-9eac-9a893e35e6df,Domestic Shipping,0.0,0.0,0.0,0.0,0.0055,2020-04-20,2018-11-06,ATI LOGISTICS,8.461538,5.0


In [320]:
## concatenate all the services and rank in 1 view

final_rank_sss= pd.concat([
    imag_rank[['provider_id','service_name','rank']],
    accounting_rank[['provider_id','service_name','rank']],
    training_rank[['provider_id','service_name','rank']],
    i_s_rank[['provider_id','service_name','rank']],
    trans_rank[['provider_id','service_name','rank']],
    i_r_rank[['provider_id','service_name','rank']],
    fba_rank[['provider_id','service_name','rank']],
    stor_rank[['provider_id','service_name','rank']],
    ipa_rank[['provider_id','service_name','rank']],
    exec_rank[['provider_id','service_name','rank']],
    cat_rank[['provider_id','service_name','rank']],
    tax_rank[['provider_id','service_name','rank']],
    comp_rank[['provider_id','service_name','rank']],
    adz_rank[['provider_id','service_name','rank']],
    am_rank[['provider_id','service_name','rank']],
    ebc_rank[['provider_id','service_name','rank']],
    snr_rank[['provider_id','service_name','rank']],
    ds_rank[['provider_id','service_name','rank']]

    ],ignore_index=True)

final_rank_sss.head()

,provider_id,service_name,rank
0,160779a5-e457-4a57-bb42-f14bbc8ea4e4,Imaging,1.0
1,a5c32bce-5fe4-4f64-8764-7987f12eab79,Imaging,2.0
2,10fddeed-85f8-4f4f-9f5c-b7ead466b0b1,Imaging,3.0
3,860468c6-9e94-4627-8e89-23e58e944fff,Imaging,4.0
4,7ef4588b-b4f9-4ec2-b162-fa820c1a8236,Imaging,5.0


In [321]:
final_rank_sss.to_clipboard()

In [322]:
## have to integrate fifo logic, table to be found and logic to be added (maybe solve this in sql where you can join with the provider master and use the data fiels in the rank)

In [323]:
## consolidate all the data frames into 1 single report
path= r'C:\Users\arvinmvn\Desktop\Ranking\ranking_in_output_python.xlsx'

writer = pd.ExcelWriter(path,engine='xlsxwriter')
# Write each dataframe to a different worksheet.
final_rank_sss.to_excel(writer, sheet_name='final_rank',header = True,index=False)
join_1_20.to_excel(writer, sheet_name='metrics_raw',header = True,index=False)
imag_rank.to_excel(writer, sheet_name='imaging',header = True,index=False)
accounting_rank.to_excel(writer, sheet_name='accounting',header = True,index=False)
training_rank.to_excel(writer, sheet_name='training',header = True,index=False)
i_s_rank.to_excel(writer, sheet_name='international_shipping',header = True,index=False)
trans_rank.to_excel(writer, sheet_name='translation',header = True,index=False)
i_r_rank.to_excel(writer, sheet_name='international retuns',header = True,index=False)
fba_rank.to_excel(writer, sheet_name='fba preparation',header = True,index=False)
stor_rank.to_excel(writer, sheet_name='storage',header = True,index=False)
ipa_rank.to_excel(writer, sheet_name='ip accelarator',header = True,index=False)
exec_rank.to_excel(writer, sheet_name='excess inventory',header = True,index=False)
cat_rank.to_excel(writer, sheet_name='cataloguing',header = True,index=False)
tax_rank.to_excel(writer, sheet_name='taxes',header = True,index=False)
comp_rank.to_excel(writer, sheet_name='compliance',header = True,index=False)
adz_rank.to_excel(writer, sheet_name='ad optimization',header = True,index=False)
am_rank.to_excel(writer, sheet_name='account management',header = True,index=False)
ebc_rank.to_excel(writer, sheet_name='ebc',header = True,index=False)
snr_rank.to_excel(writer, sheet_name='snr',header = True,index=False)
ds_rank.to_excel(writer, sheet_name='ds',header = True,index=False)
writer.save()

C:\Users\arvinmvn\AppData\Local\Temp\ipykernel_13848\402810154.py:26: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
